## Name: 
Xiaojing Shen/xshen16@depaul.edu  
Serena Yang/myang43@depaul.edu  
Yue Hou/yhou14@depaul.edu

### Group 3_Check point 2: Crawler Web

In [1]:
#download this if you don't have this package in your system
!pip install selenium
!pip install textblob

     |████████████████████████████████| 138 kB 7.1 MB/s eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

conda 4.10.3 requires ruamel_yaml_conda>=0.11.14, which is not installed.
conda-repo-cli 1.0.3 requires pathlib, which is not installed.
requests 2.24.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.9 which is incompatible.


In [2]:
import requests
from bs4 import BeautifulSoup
#from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
import pandas as pd
#from textblob import TextBlob
import time
import re
from datetime import datetime
from re import sub
from decimal import Decimal

#### get movie list from https://www.imdb.com/search/title/?groups=top_250

In [20]:
movie_dic = {}

for page in range(1, 251, 50):
    url = f"https://www.imdb.com/search/title/?groups=top_250&start={page}&ref_=adv_nxt"
    page = requests.get(url).text
    doc = BeautifulSoup(page, "html.parser")
    div = doc.find(class_="lister-list")
    contents = div.find_all(["h3"])
    for content in contents:
        name = content.find_all(["a"])
        for i in name:
            link = i.get('href')
            result = i.get_text()
            movie_dic[result] = link

movie_link = movie_dic.items()
movie_list = list(movie_link)
movie_dictionary = pd.DataFrame(movie_list)
movie_dictionary.to_csv(r'/Users/serenayang/Desktop/Group 3_Capstone Project/data/movie_dic.csv')

#class_="titleColumn"

In [21]:
movie_dic

{'The Batman': '/title/tt1877830/',
 'Spider-Man: No Way Home': '/title/tt10872600/',
 'The Dark Knight': '/title/tt0468569/',
 'Dune': '/title/tt1160419/',
 'Batman Begins': '/title/tt0372784/',
 'The Dark Knight Rises': '/title/tt1345836/',
 'The Shawshank Redemption': '/title/tt0111161/',
 'Joker': '/title/tt7286456/',
 'The Godfather': '/title/tt0068646/',
 'Avengers: Endgame': '/title/tt4154796/',
 'Interstellar': '/title/tt0816692/',
 'The Green Mile': '/title/tt0120689/',
 'Prisoners': '/title/tt1392214/',
 'Pulp Fiction': '/title/tt0110912/',
 'The Wolf of Wall Street': '/title/tt0993846/',
 'Inception': '/title/tt1375666/',
 'Se7en': '/title/tt0114369/',
 'Fight Club': '/title/tt0137523/',
 'The Lord of the Rings: The Fellowship of the Ring': '/title/tt0120737/',
 'There Will Be Blood': '/title/tt0469494/',
 'Mad Max: Fury Road': '/title/tt1392190/',
 'Django Unchained': '/title/tt1853728/',
 'Parasite': '/title/tt6751668/',
 'Inglourious Basterds': '/title/tt0361748/',
 'Aven

#### get movie info and reviews: 
movie_top: Movie_title, avg_rating, release_year, rating_num; 
reviews_top: Movie_title, user_name, rating_score, rating_time, num_helpful, review

In [101]:
def crawler(movie_dic):
    df = pd.DataFrame()
    M = [{'Movie_title': 0, 'avg_rating': 0, 'release_year':0, 'rating_num':0},]
    R = [{'Movie_title': 0, 'user_name': 0 , 'rating_score': 0, 'rating_time':0, 'num_helpful': 0 ,'review': 0},]

    #Creates DataFrame.
    df_R = pd.DataFrame(R)
    df_M = pd.DataFrame(M)
    M = {"Movie_title": [], "avg_rating": [], "release_year": [], "rating_num": []}

    for name in movie_dic:
        while True:
            try:
                #get beautiful soup result for each movie main page
                url = f"https://www.imdb.com{movie_dic[name]}"
                url_1 = url + "reviews/_ajax?ref_=undefined&paginationKey={}"
                key = ""
                R = {"Movie_title": [], "user_name": [], "rating_score": [], "rating_time":[], "num_helpful": [],"review": []}
                page = requests.get(url).text
                doc = BeautifulSoup(page, "html.parser")
                rate = doc.find(class_="AggregateRatingButton__RatingScore-sc-1ll29m0-1 iTLWoV").string #rating
                release_y = doc.find(class_="TitleBlockMetaData__ListItemText-sc-12ein40-2 jedhex").string #release yaer
                review_num = doc.find(class_="score").string                                           #review numbers

                if "K" in review_num:
                    review_num = review_num[:-1]
                    review_num = int(float(review_num)*1000)


                #for each movie review page 
                #while no more reviews loaded = false
                while True:
                    #get each movie review page 
                    response = requests.get(url_1.format(key))
                    soup = BeautifulSoup(response.content, "html.parser")

                    # Find the pagination key
                    # while no more reviews loaded = false
                    pagination_key = soup.find("div", class_="load-more-data")
                    if not pagination_key:
                        break

                    # Update the `key` variable in-order to scrape more reviews
                    key = pagination_key["data-key"]
                    user_name = soup.find_all(class_="display-name-link")
                    rating_score = soup.find_all(class_="rating-other-user-rating")
                    rating_time = soup.find_all(class_="review-date")
                    num_helpful = soup.find_all(class_="actions text-muted")
                    review = soup.find_all(class_="text show-more__control")

                    for user_name,rating_score,rating_time,num_helpful,review in zip(
                        user_name,rating_score,rating_time,num_helpful,review
                    ):

                        R["Movie_title"].append(name)
                        R["user_name"].append(user_name.get_text())
                        #transfer rating_score in to single number
                        rating_score = rating_score.get_text()
                        partitioned_string = rating_score.partition('/')
                        rating_score = partitioned_string[0]
                        R["rating_score"].append(int(rating_score))
                        #transfer rating date from 13 February 2022 into 02/2022
                        temp = rating_time.get_text()
                        rating_time = datetime.strptime(temp, '%d %B %Y').strftime('%Y-%m-%d')
                        R["rating_time"].append(rating_time)  
                        #transfer num_helpful from '1 out of 5 found this helpful.' into 0.2
                        temp = num_helpful.get_text()
                        partitioned_numerator = temp.partition('out')
                        numerator = partitioned_numerator[0]
                        numerator = Decimal(sub(r'[^\d.]', '', numerator))
                        pattern = "out of (.*?) found this helpful."
                        denominator = re.search(pattern, temp).group(1)
                        denominator = Decimal(sub(r'[^\d.]', '', denominator))
                        if (denominator == 0) or (numerator == 0):
                            num_helpful = 0
                        else: 
                            num_helpful = int(numerator)/int(denominator)
                        R["num_helpful"].append(round(num_helpful,2))
                        R["review"].append(review.get_text())

                M["Movie_title"].append(name)
                M["avg_rating"].append(rate)
                M["release_year"].append(release_y)
                M["rating_num"].append(review_num) 

                df_review = pd.DataFrame(R)
                frames_review = [df_R, df_review]
                df_R = pd.concat(frames_review)
                #print(df_R)
                df_R.to_csv(r'/Users/serenayang/Desktop/Group 3_Capstone Project/data/reviews_top.csv')

                df_movie = pd.DataFrame(M)
                frames_movie = [df, df_movie]
                df_M = pd.concat(frames_movie)
                print(name,review_num)
                df_M.to_csv(r'/Users/serenayang/Desktop/Group 3_Capstone Project/data/movies_top.csv')
                break
                
            except AttributeError:
                print("Oops!  That was AttributeError")
                break

In [102]:
crawler(movie_dic)

Once Upon a Time... In Hollywood 5600
Toy Story 4 1300
Spider-Man: Far from Home 2300
Cold War 226
John Wick: Chapter 3 - Parabellum 2400
How to Train Your Dragon: The Hidden World 707
El Camino: A Breaking Bad Movie 1800
Rocketman 1000
Alita: Battle Angel 2800
Midsommar 3600
Booksmart 787
Shazam! 2100
Fighting with My Family 396
Aladdin 2400
Ad Astra 3300
The Dirt 580
Soorarai Pottru 2100
Dara of Jasenovac 856
Shershaah 3100
Sardar Udham 893
Drishyam 2 1000
96 479
Raatchasan 547
Ayla: The Daughter of War 80
Chhichhore 721
Uri: The Surgical Strike 1000
Tumbbad 835
K.G.F: Chapter 1 1200
Vikram Vedha 183
3:10 to Yuma 678
The Bucket List 393
The Terminal 812
The Bridges of Madison County 276
Driving Miss Daisy 202
Millions 165
As Good as It Gets 591
Dances with Wolves 522
Babel 986
Cast Away 1200
Traffic 914
The Last King of Scotland 387
Blood Diamond 769
Boys Don't Cry 453
Apocalypto 1200
The Blind Side 566
Gandhi 319
Fried Green Tomatoes 219
Scent of a Woman 472
Titanic 3100
The Pursuit

#### get movie features

In [30]:
movie_dic.iloc[127]

Movie_title     The Wizard of Oz
movie_ID       /title/tt0032138/
Name: 127, dtype: object

In [31]:
df = pd.DataFrame(columns = ['Movie_title','Movie_intro', 'Genre' ,'rating','release_year','review_num'])
print(df)

Empty DataFrame
Columns: [Movie_title, Movie_intro, Genre, rating, release_year, review_num]
Index: []


In [25]:
GenreSet = set()
#for index, row in df_movie_lst.iterrows():
for index, row in movie_dic.iterrows():

    genre = []
    try:
        url = f"https://www.imdb.com{row['movie_ID']}"
        key = ""
        #data = {"Movie_title": [],"rating": [],"review_num": []} #,"release_year": []
        page = requests.get(url).text

        doc = BeautifulSoup(page, "html.parser")
        content = doc.find(class_="GenresAndPlot__TextContainerBreakpointXL-sc-cum89p-2 eqbKRZ").string #content

        genres = doc.find(class_="ipc-chip-list GenresAndPlot__GenresChipList-sc-cum89p-4 cDpOeC") #genre
        if genres is None:
            genres = doc.find(class_="ipc-chip-list GenresAndPlot__OffsetChipList-sc-cum89p-5 dZdTje") #genre
        for genre_single in genres:
            genre.append(genre_single.find(class_="ipc-chip__text").string)
            GenreSet.add(genre_single.find(class_="ipc-chip__text").string)

        rate = doc.find(class_="AggregateRatingButton__RatingScore-sc-1ll29m0-1 iTLWoV").string #rating

        release_y = doc.find(class_="TitleBlockMetaData__ListItemText-sc-12ein40-2 jedhex").string #release yaer

        review_num = doc.find(class_="score").string    #review numbers
        if "K" in review_num:
            review_num = review_num[:-1]
            review_num = int(float(review_num)*1000)
        df = df.append({'Movie_title' : row['Movie_title'], 'Movie_intro': content, 'Genre' :genre,'rating' : rate, 'release_year' : release_y, 'review_num' : review_num}, ignore_index = True)
    except:
        print(index, row)
        continue


0 Movie_title           The Batman
movie_ID       /title/tt1877830/
Name: 0, dtype: object
1 Movie_title    Spider-Man: No Way Home
movie_ID            /title/tt10872600/
Name: 1, dtype: object
2 Movie_title      The Dark Knight
movie_ID       /title/tt0468569/
Name: 2, dtype: object
3 Movie_title                 Dune
movie_ID       /title/tt1160419/
Name: 3, dtype: object
4 Movie_title        Batman Begins
movie_ID       /title/tt0372784/
Name: 4, dtype: object
5 Movie_title    The Dark Knight Rises
movie_ID           /title/tt1345836/
Name: 5, dtype: object
6 Movie_title    The Shawshank Redemption
movie_ID              /title/tt0111161/
Name: 6, dtype: object
7 Movie_title                Joker
movie_ID       /title/tt7286456/
Name: 7, dtype: object
8 Movie_title        The Godfather
movie_ID       /title/tt0068646/
Name: 8, dtype: object
9 Movie_title    Avengers: Endgame
movie_ID       /title/tt4154796/
Name: 9, dtype: object
10 Movie_title         Interstellar
movie_ID       /titl

In [26]:
df.to_csv('/Users/serenayang/Desktop/Group 3_Capstone Project/data/df_IMDb_top25_info.csv',index=False)